In [1]:
from nltk import word_tokenize
from nltk.corpus import stopwords, words
from nltk.corpus import wordnet as wn
from  spellchecker import SpellChecker
import wordninja
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sys import platform, path
import json
import csv
from time import time
from difflib import get_close_matches

### Load all the data

In [95]:
win_path = r"C:\Users\Anton\Desktop\projects\spell_checker\csv_imports\\"
mac_linux_path = "csv_imports/"

In [96]:
if platform == "linux" or platform == "linux2":
    path = mac_linux_path
elif platform == "darwin":
    path = mac_linux_path
elif platform == "win32":
    path = win_path
else:
    path = win_path

 query data

In [97]:
queries_df = pd.read_csv(path + "nlp_queries.csv")
queries_df = queries_df.drop(columns=["#"])
queries_df = queries_df.fillna("")

porn specific dictionary

In [98]:
porn_terms_df = pd.read_csv(path + "porn_words.csv")
porn_words = porn_terms_df["term"].unique()
pornwords_set_lower = {w.lower() for w in porn_words}

pornstars

In [99]:
pornstar_names_tnaflix = pd.read_csv(path + "pornstars2.csv", delimiter=";")
pornstar_names_tnaflix = pornstar_names_tnaflix.drop(columns=["Unnamed: 6", "id"])
pornstar_names_tnaflix = pornstar_names_tnaflix.rename(columns={"Unnamed: 3": "performer_category", "Unnamed: 4": "present_on", "Unnamed: 5": "ph_link"})
print(pornstar_names_tnaflix["name"].nunique(), "actors")

221946 actors


/Users/antongigele/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (0,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


studios

In [100]:
studio_names_tnaflix = pd.read_csv(path + "studios.csv", delimiter=";")
print(studio_names_tnaflix["title"].nunique(), "studios")

2502 studios


akronyms

In [101]:
with open('/Users/antongigele/Desktop/python/spell_checker/csv_imports/porn_acronyms.json') as json_file:
    acronyms_dict = json.load(json_file)

#### Concept for clustering
1. Cluster with difflib like {japanese: japonese, japanaise, jpanese, japanes, ...}
2. Some 1-Element clusters should be defragemented and spellchecked like heroinejapanese ---> heroine japanese
3. Detect False-correction like bqc ---> bbc should be bwc
4. Further complete clusters by adding words with spellchecker

#### Get a list of all single words in the query-column
#### Also retrieve stats about the data

In [102]:
split_subwords = False
single_words_set = set()
number_of_nonunique_words = 0
if split_subwords:
    for q in queries_df["query"].unique():
        number_of_nonunique_words += len(q.split())
        if split_subwords:
            sentence = q.split()
            for word in sentence:
                single_words_set.update(wordninja.split(word))
                single_words_set.update(set(q.split()))
else:
    for q in queries_df["query"].unique():
        number_of_nonunique_words += len(q.split())
        single_words_set.update(set(q.split()))

In [103]:
wordninja.split("batavia")

['batavia']

In [104]:
print("Number of queries:", f"{len(queries_df):,}")
print("Number of non-unique words:", f"{number_of_nonunique_words:,}")
print("Number of unique words:", f"{len(single_words_set):,}") 

Number of queries: 1,062,097
Number of non-unique words: 3,147,323
Number of unique words: 202,507


In [106]:
atoms_df = pd.DataFrame(list(single_words_set))
atoms_df.columns = ['term']
atoms_df.to_csv("/Users/antongigele/Desktop/python/spell_checker/csv_imports/atoms.csv", index=False)

In [83]:
single_words_list = []
for q in list(queries_df["query"].unique())[:10000]:
    single_words_list = list(dict.fromkeys(single_words_list + list(set(q.split()))))
    # list(dict.fromkeys(q))

In [84]:
single_words_list

['japanese',
 'sexmex',
 'hentai',
 'omegle',
 'anal',
 'pinay',
 'asian',
 'gacha',
 'life',
 'amateur',
 'mom',
 'teen',
 'scat',
 'lesbian',
 'mompov',
 'hardcore',
 'blowjob',
 'creampie',
 'roblox',
 'encoxada',
 'granny',
 'korean',
 'black',
 'somali',
 'forced',
 'compilation',
 'mother',
 'cumshot',
 'babe',
 'kutombana',
 'incest',
 'love',
 'story',
 'anime',
 'public',
 'girlsdoporn',
 'solo',
 'pamela',
 'rios',
 'tanzania',
 'thai',
 'mandingo',
 'bbw',
 'hairy',
 'indian',
 'interracial',
 'big',
 'ass',
 'sex',
 'mex',
 'family',
 'vintage',
 'dredd',
 '3d',
 'ometv',
 'futanari',
 'algerienne',
 'celebrity',
 'threesome',
 'mia',
 'khalifa',
 'and',
 'son',
 'uncensored',
 'flash',
 'dick',
 'minecraft',
 'agent',
 'massage',
 'arab',
 'bbc',
 'algerian',
 'iranian',
 'myanmar',
 'lesbians',
 'borja',
 'vika',
 'perverse',
 'sister',
 'brother',
 'game',
 'pregnant',
 'full',
 'movie',
 'yaoi',
 'young',
 'teenager',
 'defloration',
 'woodman',
 'chinese',
 'real',
 't

#### Concept for clustering
1. Create Cluster with difflib
2. Some 1-Element clusters should be defragemented and spellchecked like heroinejapanese ---> heroine japanese
3. Detect False-correction like bqc ---> bbc should be bwc
4. Further complete clusters by adding words with spellchecker

#### Create word-clusters
e.g.: japanese <--- {japonese, japanaise, jpanese, japanes, ...}

In [86]:
term = "japanese" 
term_relatives = get_close_matches(term, list(single_words_set), n=500, cutoff=0.85)
len(term_relatives)
term_relatives

161

['japanese',
 'tjapanese',
 'njapanese',
 'mjapanese',
 'jyapanese',
 'jjapanese',
 'jaqpanese',
 'jappanese',
 'japoanese',
 'japnanese',
 'japeanese',
 'japbanese',
 'japasnese',
 'japapnese',
 'japansese',
 'japannese',
 'japaniese',
 'japanesse',
 'japanesex',
 'japanesew',
 'japaneses',
 'japanesen',
 'japanesei',
 'japanesee',
 'japanesed',
 'japanese7',
 'japanesde',
 'japanesae',
 'japaneese',
 'japanease',
 'japanaese',
 'japaenese',
 'japaanese',
 'jap9anese',
 'jaopanese',
 'janpanese',
 'jaapanese',
 'ijapanese',
 'hjapanese',
 'ajapanese',
 '3japanese',
 '2japanese',
 '0japanese',
 'jpanese',
 'japnese',
 'japanse',
 'japanes',
 'japanee',
 'japaese',
 'jaanese',
 'apanese',
 'vrjapanese',
 'tvjapanese',
 'stjapanese',
 'jkjapanese',
 'japasenese',
 'japapanese',
 'japannesse',
 'japanesesm',
 'japanesese',
 'japanesemy',
 'japaneseko',
 'japanesejk',
 'japanesejd',
 'japaneseja',
 'japaneseiv',
 'japanesehd',
 'japanesedp',
 'japaneseav',
 'japanese60',
 'japanese18',
 'j

In [30]:
atoms_clusters_dict = {}
all_atoms_list = list(single_words_set)[:100]
for atom in all_atoms_list:
    atoms_clusters_dict[atom] = get_close_matches(atom, list(single_words_set), n=500, cutoff=0.85)

In [88]:
atoms_clusters_dict = {}
all_atoms_list = single_words_list[:5000]
single_words_set_test = single_words_set
for atom in all_atoms_list:
    related_words_list = get_close_matches(atom, list(single_words_set_test), n=500, cutoff=0.85)
    atoms_clusters_dict[atom] = related_words_list
    single_words_set_test = single_words_set_test - set(related_words_list)

In [89]:
atoms_clusters_dict

{'japanese': ['japanese',
  'tjapanese',
  'njapanese',
  'mjapanese',
  'jyapanese',
  'jjapanese',
  'jaqpanese',
  'jappanese',
  'japoanese',
  'japnanese',
  'japeanese',
  'japbanese',
  'japasnese',
  'japapnese',
  'japansese',
  'japannese',
  'japaniese',
  'japanesse',
  'japanesex',
  'japanesew',
  'japaneses',
  'japanesen',
  'japanesei',
  'japanesee',
  'japanesed',
  'japanese7',
  'japanesde',
  'japanesae',
  'japaneese',
  'japanease',
  'japanaese',
  'japaenese',
  'japaanese',
  'jap9anese',
  'jaopanese',
  'janpanese',
  'jaapanese',
  'ijapanese',
  'hjapanese',
  'ajapanese',
  '3japanese',
  '2japanese',
  '0japanese',
  'jpanese',
  'japnese',
  'japanse',
  'japanes',
  'japanee',
  'japaese',
  'jaanese',
  'apanese',
  'vrjapanese',
  'tvjapanese',
  'stjapanese',
  'jkjapanese',
  'japasenese',
  'japapanese',
  'japannesse',
  'japanesesm',
  'japanesese',
  'japanesemy',
  'japaneseko',
  'japanesejk',
  'japanesejd',
  'japaneseja',
  'japaneseiv',


Total sum of all clusters content

In [6]:
with open('/Users/antongigele/Desktop/python/spell_checker/csv_imports/atom_clusters.json') as json_file:
    atoms_clusters_dict = json.load(json_file)

In [7]:
i = 0
for ele in atoms_clusters_dict.values():
    i += len(ele)
i

52539

In [93]:
with open("/Users/antongigele/Desktop/python/spell_checker/csv_imports/atom_clusters.json", "w") as outfile: 
        json.dump(atoms_clusters_dict, outfile)

Remove stopwords from queries

In [17]:
stop_words = set(stopwords.words('english')) 
def remove_stop_words(sentence): 
    words = sentence.split() 
    filtered_words = [word for word in words if word not in stop_words] 

    return ' '.join(filtered_words)

#### Feed the dictionary specific porn-terms
* Pornstars names
* Studios names
* List of movies and series (Focus on games and animes)
* Akronyms
* proper names

Proper names aka porn-neologisms

In [18]:
proper_porn_words = ['hentai', 'omegle', 'sexmex', 'roblox', 'pinay', 'futanari', 'kodak']

In [19]:
spell = SpellChecker()

In [20]:
spell.word_frequency.load_words(list(pornstar_names_tnaflix["name"].unique()))
spell.word_frequency.load_words(list(studio_names_tnaflix["title"].unique()))
spell.word_frequency.load_words(list(acronyms_dict.keys()))
spell.word_frequency.load_words(proper_porn_words)

In [21]:
def spellcheck_porn(query):
    # start = time()
    query = remove_stop_words(query)
    query_list = word_tokenize(query)
    mispelled = spell.unknown(query_list)
    if len(mispelled) == 0:
        # end = time()
        # print(end-start)
        return query_list
    else:
        result_list = [w for w in query_list if w not in mispelled]
        for word in mispelled:
            # candidates = spell.candidates(word)
            if spell.candidates(word) == None:
                word = " ".join(wordninja.split(word))
                result_list += spellcheck_porn(word)
            elif len(list(set(spell.candidates(word)).intersection(pornwords_set_lower))) == 0:
                result_list.append(spell.correction(word))
            else:
                result_list.append(list(set(spell.candidates(word)).intersection(pornwords_set_lower))[0])
        # end = time()
        # print(end-start)
        return result_list

In [22]:
split_words = wordninja.split("foodsex")
split_words_str = " ".join(split_words)

In [23]:
split_words_str

'food sex'

In [24]:
word_tokenize(split_words_str)

['food', 'sex']

In [25]:
spellcheck_porn("kodak")

['kodak']

In [26]:
strange_words = []
known_words = []
for word in list(single_words_set)[:10]:
    corrected_word = spellcheck_porn(word)
    print(word, corrected_word)
    known_words.append(corrected_word)

unova ['nova']
ginavalentina ['gina valentina']
aloukasquirt ['alo', 'uk', 'squirt']
katorseanyos ['kat', 'sea', 'no']
chehian ['chemin']
publicdisgrace ['public disgrace']
ugotitflauntit ['u', 'got', 'flaunt']
uncorced ['uncorked']
ozeex ['zoeex']
stock ['stock']


In [27]:
word = "kodak" 
for q in  queries_df["query"].unique():
    if word in q:
        print(q)

gqueen riho kodaka pose soro
kodak robert agency that was awesome
ford estonia kodak
kodak black


#### Detect false-correction
bqc ------> bwc, but got corrected to "bbc" because of the higher frequency of "bbc"

In [35]:
for word in spellcheck_porn("bqc"):
    print(word)

{'bic', 'bbc', 'bac', 'bwc', 'bec'}
bbc


example of queries belonging to the same cluster

In [23]:
query_1 = "japanese milf with big tit"
query_2 = "japanese milf big tits"

In [25]:
spellcheck_porn(query_2)

['japanese', 'milf', 'big', 'tits']